In [1]:
import pandas as pd
import requests
import numpy as np
import time
import geopandas as gp
import matplotlib
import matplotlib.pyplot as plt
import lxml
import os
import glob
from shapely.geometry import Point
from adjustText import adjust_text
from bs4 import BeautifulSoup
from tqdm import tqdm
from tqdm import trange
from datetime import datetime

%matplotlib inline

In [2]:
## CAMADA TRUSTED

In [3]:
import pandas as pd

df = gp.read_file('C:/Users/crash/Desktop/raw_speedtest_fixed.shp')

In [4]:
df.head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,index_righ,CD_MUN,fonte,geometry
0,2110223123011000,157341,56947,13,1095,259,3174,3205309,https://ookla-open-data.s3-us-west-2.amazonaws...,"POLYGON ((-40.29785 -20.30342, -40.29236 -20.3..."
1,2110223121232032,184049,107142,4,120,45,3174,3205309,https://ookla-open-data.s3-us-west-2.amazonaws...,"POLYGON ((-40.33081 -20.27766, -40.32532 -20.2..."
2,2110223123001110,176971,90063,23,221,52,3174,3205309,https://ookla-open-data.s3-us-west-2.amazonaws...,"POLYGON ((-40.35278 -20.30342, -40.34729 -20.3..."
3,2110223121223330,160729,94449,5,105,26,3174,3205309,https://ookla-open-data.s3-us-west-2.amazonaws...,"POLYGON ((-40.35278 -20.29311, -40.34729 -20.2..."
4,2110223121233110,167756,63182,9,590,119,3174,3205309,https://ookla-open-data.s3-us-west-2.amazonaws...,"POLYGON ((-40.26489 -20.26220, -40.25940 -20.2..."


In [5]:
df['latitude'] = df.geometry.centroid.y
df['longitude'] = df.geometry.centroid.x
df.head()

C:\Users\crash\AppData\Local\Temp\ipykernel_18140\2334890284.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df['latitude'] = df.geometry.centroid.y
C:\Users\crash\AppData\Local\Temp\ipykernel_18140\2334890284.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df['longitude'] = df.geometry.centroid.x


,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,index_righ,CD_MUN,fonte,geometry,latitude,longitude
0,2110223123011000,157341,56947,13,1095,259,3174,3205309,https://ookla-open-data.s3-us-west-2.amazonaws...,"POLYGON ((-40.29785 -20.30342, -40.29236 -20.3...",-20.305993,-40.295105
1,2110223121232032,184049,107142,4,120,45,3174,3205309,https://ookla-open-data.s3-us-west-2.amazonaws...,"POLYGON ((-40.33081 -20.27766, -40.32532 -20.2...",-20.280232,-40.328064
2,2110223123001110,176971,90063,23,221,52,3174,3205309,https://ookla-open-data.s3-us-west-2.amazonaws...,"POLYGON ((-40.35278 -20.30342, -40.34729 -20.3...",-20.305993,-40.350037
3,2110223121223330,160729,94449,5,105,26,3174,3205309,https://ookla-open-data.s3-us-west-2.amazonaws...,"POLYGON ((-40.35278 -20.29311, -40.34729 -20.2...",-20.295690,-40.350037
4,2110223121233110,167756,63182,9,590,119,3174,3205309,https://ookla-open-data.s3-us-west-2.amazonaws...,"POLYGON ((-40.26489 -20.26220, -40.25940 -20.2...",-20.264774,-40.262146


In [6]:
# verificar se alguma coluna tem valores nulos
print(df.isnull().any())

quadkey       False
avg_d_kbps    False
avg_u_kbps    False
avg_lat_ms    False
tests         False
devices       False
index_righ    False
CD_MUN        False
fonte         False
geometry      False
latitude      False
longitude     False
dtype: bool


In [7]:
# contar o número de valores nulos em cada coluna
print(df.isnull().sum())

quadkey       0
avg_d_kbps    0
avg_u_kbps    0
avg_lat_ms    0
tests         0
devices       0
index_righ    0
CD_MUN        0
fonte         0
geometry      0
latitude      0
longitude     0
dtype: int64


In [8]:
# quantidade total de linhas
print(df.shape)

(1807570, 12)


In [9]:
# todos os campos numericos já são do tipo inteiro, com execção do kilometro

In [10]:
# df = gp.read_file('C:/Users/crash/Desktop/raw_ingestao_mobile.shp')

df["date"] = df["fonte"].str[105:115]
df["quarter"] = df["fonte"].str[103:104]
df = df.drop('fonte', axis=1)
# df = df.drop('Unnamed: 0', axis=1)
# df['avg_d_mbps'] = df['avg_d_kbps'] / 1024
# df['avg_d_gbps'] = df['avg_d_mbps'] / 1024
# df['avg_u_mbps'] = df['avg_u_kbps'] / 1024
# df['avg_u_gbps'] = df['avg_u_mbps'] / 1024
df['network_type'] = 'fixed'
df.head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,index_righ,CD_MUN,geometry,latitude,longitude,date,quarter,network_type
0,2110223123011000,157341,56947,13,1095,259,3174,3205309,"POLYGON ((-40.29785 -20.30342, -40.29236 -20.3...",-20.305993,-40.295105,2022-01-01,1,fixed
1,2110223121232032,184049,107142,4,120,45,3174,3205309,"POLYGON ((-40.33081 -20.27766, -40.32532 -20.2...",-20.280232,-40.328064,2022-01-01,1,fixed
2,2110223123001110,176971,90063,23,221,52,3174,3205309,"POLYGON ((-40.35278 -20.30342, -40.34729 -20.3...",-20.305993,-40.350037,2022-01-01,1,fixed
3,2110223121223330,160729,94449,5,105,26,3174,3205309,"POLYGON ((-40.35278 -20.29311, -40.34729 -20.2...",-20.295690,-40.350037,2022-01-01,1,fixed
4,2110223121233110,167756,63182,9,590,119,3174,3205309,"POLYGON ((-40.26489 -20.26220, -40.25940 -20.2...",-20.264774,-40.262146,2022-01-01,1,fixed


In [11]:
df.to_file('trusted_speedtest_fixed.shp', driver='ESRI Shapefile')

C:\Users\crash\AppData\Local\Temp\ipykernel_18140\464353810.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  df.to_file('trusted_speedtest_fixed.shp', driver='ESRI Shapefile')
